Baseball Prediction: 1 - Data Wrangling

In this notebook, we will wrangle data downloaded from www.retrosheet.org into a dataframe suitable for model building. Specifically, for each game, we will calculate some team statistics over their past 162 games.

At the end, we save our dataframe to a file. This file will be the starting point for the next notebook, in which we build our first model.

To use this notebook, you must first download the game logs here: https://www.retrosheet.org/gamelogs/index.html Towards the bottom of the page there are links for ZIP files containing multiple seasons. Download the zip files containing the 'gamelog' data, decompress it, and then move all of the single season files to a single directory. You will then need the path to that directory for the variable fname below.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)

In [2]:
fname = '/Users/antiprotons/Downloads/alldata/gamelogs/'+'gl2022.txt'
df = pd.read_csv(fname, header=None)

In [3]:
df.head(25)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160
0,20220407,0,Thu,SDN,NL,1,ARI,NL,1,2,4,51,N,NaN,NaN,NaN,PHO01,35508.0,198,001100000,000000004,29,4,1,0,0,2,0,0,0,7,0,5,1,0,2,0,7,5,4,4,2,0,24,8,0,0,1,0,26,3,0,0,1,3,0,0,1,7,0,6,0,0,1,0,6,6,2,2,0,0,27,9,0,0,2,0,vanol901,Larry Vanover,belld901,Dan Bellino,barbs901,Sean Barber,valej901,Junior Valentine,NaN,(none),NaN,(none),melvb001,Bob Melvin,lovut001,Tony Lovullo,mantj002,Joe Mantiply,suarr002,Robert Suarez,NaN,(none),beers001,Seth Beer,darvy001,Yu Darvish,bumgm001,Madison Bumgarner,nolaa002,Austin Nola,2,machm001,Manny Machado,5,cronj001,Jake Cronenworth,4,voitl001,Luke Voit,10,myerw001,Wil Myers,9,hosme001,Eric Hosmer,3,profj001,Jurickson Profar,7,kim-h002,Ha-Seong Kim,6,grist001,Trent Grisham,8,varsd001,Daulton Varsho,8,martk001,Ketel Marte,4,perad001,David Peralta,7,walkc002,Christian Walker,3,smitp002,Pavin Smith,9,kellc002,Carson Kelly,2,beers001,Seth Beer,10,ellid002,Drew Ellis,5,perdg001,Gerardo Perdomo,6,NaN,Y
1,20220407,0,Thu,CIN,NL,1,ATL,NL,1,6,3,54,N,NaN,NaN,NaN,ATL03,40545.0,181,012003000,001000020,35,10,0,0,1,6,0,1,1,1,0,13,0,0,1,0,5,5,2,2,0,0,27,7,1,0,0,0,31,4,0,0,1,2,0,0,0,5,0,13,0,0,0,0,6,4,6,6,0,0,27,7,0,0,1,0,laynj901,Jerry Layne,wendh902,Hunter Wendelstedt,whitc901,Chad Whitson,hamaa901,Adam Hamari,NaN,(none),NaN,(none),belld002,David Bell,snitb801,Brian Snitker,mahlt001,Tyler Mahle,friem001,Max Fried,santt001,Tony Santillan,farmk001,Kyle Farmer,mahlt001,Tyler Mahle,friem001,Max Fried,indij001,Jonathan India,4,aquia001,Aristides Aquino,9,phamt001,Tommy Pham,7,vottj001,Joey Votto,3,stept001,Tyler Stephenson,2,senzn001,Nick Senzel,8,mousm001,Mike Moustakas,10,farmk001,Kyle Farmer,6,drurb001,Brandon Drury,5,rosae001,Eddie Rosario,9,olsom001,Matt Olson,3,rilea001,Austin Riley,5,ozunm001,Marcell Ozuna,7,albio001,Ozzie Albies,4,duvaa001,Adam Duvall,8,darnt001,Travis d'Arnaud,2,dicka001,Alex Dickerson,10,swand001,Dansby Swanson,6,NaN,Y
2,20220407,0,Thu,MIL,NL,1,CHN,NL,1,4,5,51,D,NaN,NaN,NaN,CHI11,35112.0,198,000100210,00003020x,33,10,4,0,0,4,0,2,1,4,0,9,1,0,2,0,9,3,5,5,0,0,24,10,0,0,1,0,29,8,3,0,1,5,0,1,1,4,0,7,0,1,1,0,6,6,4,4,0,0,27,9,0,0,2,0,barrt901,Ted Barrett,barkl901,Lance Barksdale,lentn901,Nic Lentz,cejan901,Nestor Ceja,NaN,(none),NaN,(none),counc001,Craig Counsell,rossd001,David Ross,givem001,Mychal Givens,ashba003,Aaron Ashby,robed002,David Robertson,happi001,Ian Happ,burnc002,Corbin Burnes,hendk001,Kyle Hendricks,wongk001,Kolten Wong,4,adamw002,Willy Adames,6,yelic001,Christian Yelich,7,mccua001,Andrew McCutchen,10,tellr001,Rowdy Tellez,3,renfh001,Hunter Renfroe,9,narvo001,Omar Narvaez,2,cainl001,Lorenzo Cain,8,petej002,Jace Peterson,5,orter001,Rafael Ortega,10,madrn001,Nick Madrigal,4,contw001,Willson Contreras,2,happi001,Ian Happ,7,schwf001,Frank Schwindel,3,suzus001,Seiya Suzuki,9,heywj001,Jason Heyward,8,wisdp001,Patrick Wisdom,5,hoern001,Nico Hoerner,6,NaN,Y
3,20220407,0,Thu,PIT,NL,1,SLN,NL,1,0,9,51,D,NaN,NaN,NaN,STL10,46256.0,188,000000000,13000104x,30,6,0,0,0,0,0,0,0,2,0,9,0,1,2,0,5,6,9,9,0,0,24,7,2,0,1,0,31,8,2,0,3,9,0,2,1,7,0,5,1,0,1,0,8,4,0,0,1,0,27,11,0,0,3,0,reynj901,Jim Reynolds,ticht901,Todd Tichenor,muchm901,Mike Muchlinski,merzd901,Dan Merzel,NaN,(none),NaN,(none),sheld801,Derek Shelton,marmo801,Oliver Marmol,waina001,Adam Wainwright,brubj001,JT Brubaker,NaN,(none),oneit001,Tyler O'Neill,brubj001,JT Brubaker,waina001,Adam Wainwright,voged001,Daniel Vogelbach,10,reynb001,Bryan Reynolds,8,hayek001,Ke'Bryan Hayes,5,tsuty001,Yoshi Tsutsugo,3,newmk001,Kevin Newman,6,tu

In [4]:
colnames = ['date','dblheader_code','day_of_week','team_v','league_v','game_no_v',
           'team_h','league_h','game_no_h', 'runs_v', 'runs_h','outs_total','day_night',
            'completion_info','forfeit_info','protest_info','ballpark_id','attendance','game_minutes',
            'linescore_v','linescore_h',
           'AB_v','H_v','2B_v','3B_v','HR_v','RBI_v','SH_v','SF_v','HBP_v','BB_v','IBB_v','SO_v',
            'SB_v', 'CS_v','GIDP_v','CI_v','LOB_v',
            'P_num_v','ERind_v','ERteam_v','WP_v','balk_v',
            'PO_v','ASST_v','ERR_v','PB_v','DP_v','TP_v',
           'AB_h', 'H_h', '2B_h', '3B_h', 'HR_h', 'RBI_h', 'SH_h', 'SF_h', 'HBP_h', 'BB_h', 'IBB_h','SO_h',
            'SB_h', 'CS_h', 'GIDP_h', 'CI_h', 'LOB_h',
            'P_num_h', 'ERind_h', 'ERteam_h', 'WP_h', 'balk_h',
            'PO_h', 'ASST_h', 'ERR_h', 'PB_h', 'DP_h', 'TP_h',
            'ump_HB_id', 'ump_HB_name','ump_1B_id', 'ump_1B_name','ump_2B_id', 'ump_2B_name',
            'ump_3B_id', 'ump_3B_name','ump_LF_id', 'ump_LF_name','ump_RF_id', 'ump_RF_name',
            'mgr_id_v', 'mgr_name_v', 'mgr_id_h', 'mgr_name_h',
            'pitcher_id_w','pitcher_name_w','pitcher_id_l','pitcher_name_l','pitcher_id_s','pitcher_name_s',
            'GWRBI_id','GWRBI_name','pitcher_start_id_v','pitcher_start_name_v','pitcher_start_id_h','pitcher_start_name_h',
            'batter1_name_v', 'batter1_id_v', 'batter1_pos_v', 'batter2_name_v', 'batter2_id_v', 'batter2_pos_v',
            'batter3_name_v', 'batter3_id_v', 'batter3_pos_v', 'batter4_name_v', 'batter4_id_v', 'batter4_pos_v',
            'batter5_name_v', 'batter5_id_v', 'batter5_pos_v', 'batter6_name_v', 'batter6_id_v', 'batter6_pos_v',
            'batter7_name_v', 'batter7_id_v', 'batter7_pos_v', 'batter8_name_v', 'batter8_id_v', 'batter8_pos_v',
            'batter9_name_v', 'batter9_id_v', 'batter9_pos_v', 'batter1_name_h', 'batter1_id_h', 'batter1_pos_h',
            'batter2_name_h', 'batter2_id_h', 'batter2_pos_h', 'batter3_name_h', 'batter3_id_h', 'batter3_pos_h',
            'batter4_name_h', 'batter4_id_h', 'batter4_pos_h', 'batter5_name_h', 'batter5_id_h', 'batter5_pos_h',
            'batter6_name_h', 'batter6_id_h', 'batter6_pos_h', 'batter7_name_h', 'batter7_id_h', 'batter7_pos_h',
            'batter8_name_h', 'batter8_id_h', 'batter8_pos_h', 'batter9_name_h', 'batter9_id_h', 'batter9_pos_h',           
           'misc_info','acqui_info'
           ]

df.columns = colnames

df.sample(25)

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info
361,20220504,0,Wed,MIN,AL,25,BAL,AL,25,4,9,51,N,NaN,NaN,NaN,BAL12,7466.0,163,000400000,00630000x,28,8,0,0,1,4,0,1,1,4,0,6,0,0,4,0,3,3,9,9,0,0,24,4,2,0,1,0,34,12,2,0,2,9,0,1,0,2,0,8,0,0,1,0,4,4,4,4,0,0,27,12,0,0,5,0,wegnm901,Mark Wegner,rehaj901,Jeremie Rehak,wolcq901,Quinn Wolcott,porta901,Alan Porter,NaN,(none),NaN,(none),baldr001,Rocco Baldelli,hydeb801,Brandon Hyde,perec004,Cionel Perez,bundd001,Dylan Bundy,NaN,(none),mullc002,Cedric Mullins,bundd001,Dylan Bundy,bradk001,Kyle Bradish,arral001,Luis Arraez,4,corrc001,Carlos Correa,6,polaj001,Jorge Polanco,10,larnt001,Trevor Larnach,9,miraj002,Jose Miranda,3,gordn001,Nick Gordon,7,sancg002,Gary Sanchez,2,urshg001,Giovanny Urshela,5,celeg001,Gilberto Celestino,8,mullc002,Cedric Mullins,8,manct001,Trey Mancini,10,santa003,Anthony Santander,9,haysa001,Austin Hays,7,mounr001,Ryan Mountcastle,3,odorr001,Rougned Odor,4,uriar001,Ramon Urias,5,chirr001,Robinson Chirinos,2,matej003,Jorge Mateo,6,NaN,Y
2303,20220927,0,Tue,ATL,NL,155,WAS,NL,154,8,2,54,N,NaN,NaN,NaN,WAS11,23281.0,167,000222110,110000000,38,10,1,1,3,7,0,0,0,3,0,8,0,0,0,0,6,5,2,2,0,0,27,7,0,0,0,0,33,6,2,0,1,2,0,0,0,1,0,11,1,0,0,0,5,5,7,7,1,0,27,10,1,0,0,0,wendh902,Hunter Wendelstedt,whitc901,Chad Whitson,hallt901,Tom Hallion,hamaa901,Adam Hamari,NaN,(none),NaN,(none),snitb801,Brian Snitker,martd002,Dave Martinez,mchuc001,Collin McHugh,espip001,Paolo Espino,NaN,(none),arcio002,Orlando Arcia,mullk001,Kyle Muller,espip001,Paolo Espino,acunr001,Ronald Acuna,9,swand001,Dansby Swanson,6,rilea001,Austin Riley,5,olsom001,Matt Olson,3,harrm004,Michael Harris,8,contw002,William Contreras,2,rosae001,Eddie Rosario,7,ozunm001,Marcell Ozuna,10,arcio002,Orlando Arcia,4,thoml002,Lane Thomas,9,abrac001,C. J. Abrams,6,menej001,Joey Meneses,3,voitl001,Luke Voit,10,calla002,Alex Call,7,hernc005,Cesar Hernandez,4,vargi001,Ildemaro Vargas,5,roblv001,Victor Robles,8,barrt004,Tres Barrera,2,NaN,Y
817,20220607,0,Tue,WAS,NL,57,MIA,NL,53,2,12,51,N,NaN,NaN,NaN,MIA02,7112.0,177,000001010,05052000x,35,8,3,0,1,2,0,0,1,2,0,6,1,0,0,0,9,4,12,12,0,0,24,9,0,0,0,0,38,14,2,0,4,12,0,0,0,4,0,9,1,0,0,0,6,4,2,2,0,0,27,11,1,0,0,0,lentn901,Nic Lentz,cejan901,Nestor Ceja,barrt901,Ted Barrett,barkl901,Lance Barksdale,NaN,(none),NaN,(none),martd002,Dave Martinez,mattd001,Don Mattingly,cabre003,Edward Cabrera,adonj001,Joan Adon,

CREATE DATA FRAME CONTAINING THE GAME LOG FOR EVERY GAME SINCE 1970 (1970-PRESENT)

In [5]:
df = pd.DataFrame()
for year in range(1970,2023):
    fname = '/Users/antiprotons/Downloads/alldata/gamelogs/gl' +str(year)+'.txt'
    df_temp = pd.read_csv(fname, header=None)
    df_temp.columns = colnames
    df_temp['season'] = year
    df = pd.concat((df, df_temp))

In [6]:
df.shape

(116082, 162)

In [7]:
df.info(max_cols=200)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116082 entries, 0 to 2429
Data columns (total 162 columns):
 #    Column                Non-Null Count   Dtype  
---   ------                --------------   -----  
 0    date                  116082 non-null  int64  
 1    dblheader_code        116082 non-null  int64  
 2    day_of_week           116082 non-null  object 
 3    team_v                116082 non-null  object 
 4    league_v              116082 non-null  object 
 5    game_no_v             116082 non-null  int64  
 6    team_h                116082 non-null  object 
 7    league_h              116082 non-null  object 
 8    game_no_h             116082 non-null  int64  
 9    runs_v                116082 non-null  int64  
 10   runs_h                116082 non-null  int64  
 11   outs_total            116082 non-null  int64  
 12   day_night             116082 non-null  object 
 13   completion_info       116 non-null     object 
 14   forfeit_info          5 non-null    

In [8]:
## Calculate a few useful columns

df['run_diff'] = df['runs_h']-df['runs_v']
df['home_victory'] = (df['run_diff']>0).astype(int)
df['run_total'] = df['runs_h'].copy()+df['runs_v'].copy()
df['date_dblhead'] = (df['date'].astype(str) + df['dblheader_code'].astype(str)).astype(int)

In [9]:
df.home_victory.mean()

0.5382057511069761

Big picture - have outcome, need features

- Would like to start simple, base features on team results over past n games
- Need to do a bit of "data wrangling"
- First, we need to have each team's games in chronological order
- Then can use pandas 'rolling' functionality to aggregate over windows in the past

- Immediate goal, get batting average, on-base percentage, and slugging percentage
- Need "running values" based on the past, not including the current game

In [10]:
# Let's look at a single team's games

df_reds = df.loc[((df.team_v=='CIN') | (df.team_h=='CIN')), :]
df_reds.shape

(8359, 166)

In [11]:
df_reds.sample(50)

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead
1444,19970728,0,Mon,CIN,NL,103,FLO,NL,103,4,0,54,N,NaN,NaN,NaN,MIA01,18393.0,167.0,000000004,000000000,31.0,7.0,1.0,0.0,2.0,4.0,0.0,0.0,1.0,5.0,1.0,11.0,1.0,3.0,0.0,0.0,6.0,3.0,0.0,0.0,1.0,0.0,27.0,10.0,1.0,0.0,2.0,0.0,33.0,8.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8.0,2.0,0.0,1.0,0.0,7.0,3.0,4.0,4.0,0.0,0.0,27.0,9.0,0.0,0.0,1.0,0.0,pullf901,Frank Pulli,herna901,Angel Hernandez,bonig901,Greg Bonin,grege901,Eric Gregg,NaN,(none),NaN,(none),mckej801,Jack McKeon,leylj801,Jim Leyland,belis001,Stan Belinda,powej001,Jay Powell,NaN,(none),olivj001,Joe Oliver,smilj001,John Smiley,leita001,Al Leiter,sandd001,Deion Sanders,8.0,reesp001,Pokey Reese,6.0,sandr002,Reggie Sanders,9.0,peree001,Eduardo Perez,3.0,greew001,Willie Greene,5.0,olivj001,Joe Oliver,2.0,kellm001,Mike Kelly,7.0,boonb002,Bret Boone,4.0,smilj001,John Smiley,1.0,whitd001,Devon White,8.0,rente001,Edgar Renteria,6.0,abbok002,Kurt Abbott,4.0,shefg001,Gary Sheffield,9.0,bonib001,Bobby Bonilla,5.0,aloum001,Moises Alou,7.0,conij001,Jeff Conine,3.0,johnc002,Charles Johnson,2.0,leita001,Al Leiter,1.0,NaN,Y,1997,-4,0,4,199707280
769,20190526,0,Sun,CIN,NL,52,CHN,NL,51,10,2,54,D,NaN,NaN,NaN,CHI11,40884.0,207.0,201004012,000001010,40.0,17.0,3.0,0.0,1.0,7.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,1.0,2.0,0.0,9.0,4.0,2.0,2.0,0.0,0.0,27.0,8.0,1.0,0.0,1.0,0.0,34.0,8.0,0.0,0.0,2.0,2.0,0.0,0.0,1.0,3.0,0.0,13.0,0.0,0.0,0.0,0.0,9.0,4.0,8.0,8.0,1.0,0.0,27.0,13.0,2.0,0.0,2.0,0.0,eddid901,Doug Eddings,millb901,Bill Miller,conrc901,Chris Conroy,rehaj901,Jeremie Rehak,NaN,(none),NaN,(none),belld002,David Bell,maddj801,Joe Maddon,roart001,Tanner Roark,quinj001,Jose Quintana,NaN,(none),suare001,Eugenio Suarez,roart001,Tanner Roark,quinj001,Jose Quintana,senzn001,Nick Senzel,8.0,vottj001,Joey Votto,3.0,suare001,Eugenio Suarez,5.0,puigy001,Yasiel Puig,9.0,iglej001,Jose Iglesias,6.0,winkj002,Jesse Winker,7.0,peraj003,Jose Peraza,4.0,casac001,Curt Casali,2.0,roart001,Tanner Roark,1.0,schwk001,Kyle Schwarber,7.0,bryak001,Kris Bryant,9.0,rizza001,Anthony Rizzo,3.0,baezj001,Javier Baez,6.0,contw001,Willson Contreras,2.0,heywj001,Jason Heyward,8.0,boted002,David Bote,5.0,descd001,Daniel Descalso,4.0,quinj001,Jose Quintana,1.0,NaN,Y,2019,-8,0,12,201905260
428,20190430,0,Tue,CIN,NL,29,NYN,NL,29,3,4,58,N,NaN,NaN,NaN,NYC20,20836.0

In [12]:
# Write a function to create a team-specific data frame, given the team
def strip_suffix(x, suff):
    if x.endswith(suff):
        return(x[:-len(suff)])
    else:
        return(x)

visit_cols = [col for col in df.columns if not col.endswith('_h')]
visit_cols_stripped = [strip_suffix(col,'_v') for col in visit_cols]
home_cols = [col for col in df.columns if not col.endswith('_v')]
home_cols_stripped = [strip_suffix(col,'_h') for col in home_cols]

## This subsets the game level df by team, to aggregate team statistics easily
## We also create rolling sums with an offset, so that the rollsum number represents
## statistics up to, but not including, the game in question

def create_team_df(team):
    df_team_v = df[(df.team_v==team)]
    opponent = df_team_v['team_h']
    df_team_v = df_team_v[visit_cols]
    df_team_v.columns = visit_cols_stripped
    df_team_v['home_game'] = 0
    df_team_v['opponent'] = opponent

    df_team_h = df[(df.team_h==team)]
    opponent = df_team_h['team_v']
    df_team_h = df_team_h[home_cols]
    df_team_h.columns = home_cols_stripped
    df_team_h['home_game'] = 1
    df_team_h['opponent'] = opponent


    df_team = pd.concat((df_team_h, df_team_v))
    df_team.sort_values(['date', 'game_no'],inplace=True)
    
    for winsize in [162,30]:
        suff = str(winsize)
        for raw_col in ['AB','H','2B','3B','HR','BB','runs','SB','CS','ERR']:
            new_col = 'rollsum_'+raw_col+'_'+suff
            df_team[new_col] = df_team[raw_col].rolling(winsize, closed='left').sum()

        df_team['rollsum_BATAVG_'+suff] = df_team['rollsum_H_'+suff] / df_team['rollsum_AB_'+suff]
        df_team['rollsum_OBP_'+suff] = (df_team['rollsum_H_'+suff] + df_team['rollsum_BB_'+suff]) / (
                                    df_team['rollsum_AB_'+suff]+df_team['rollsum_BB_'+suff])
        df_team['rollsum_SLG_'+suff] = (df_team['rollsum_H_'+suff] + df_team['rollsum_2B_'+suff] 
                                 + 2*df_team['rollsum_3B_'+suff]+ 
                                3*df_team['rollsum_HR_'+suff] ) / (df_team['rollsum_AB_'+suff])
        df_team['rollsum_OBS_'+suff] = df_team['rollsum_OBP_'+suff] + df_team['rollsum_SLG_'+suff]
    
    df_team['season_game'] = df_team['season']*1000 + df_team['game_no']
    df_team.set_index('season_game', inplace=True)
    return(df_team)
df_reds = create_team_df('CIN')
df_reds.sample(10)

,date,dblheader_code,day_of_week,team,league,game_no,runs,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore,AB,H,2B,3B,HR,RBI,SH,SF,HBP,BB,IBB,SO,SB,CS,GIDP,CI,LOB,P_num,ERind,ERteam,WP,balk,PO,ASST,ERR,PB,DP,TP,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id,mgr_name,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id,pitcher_start_name,batter1_name,batter1_id,batter1_pos,batter2_name,batter2_id,batter2_pos,batter3_name,batter3_id,batter3_pos,batter4_name,batter4_id,batter4_pos,batter5_name,batter5_id,batter5_pos,batter6_name,batter6_id,batter6_pos,batter7_name,batter7_id,batter7_pos,batter8_name,batter8_id,batter8_pos,batter9_name,batter9_id,batter9_pos,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,home_game,opponent,rollsum_AB_162,rollsum_H_162,rollsum_2B_162,rollsum_3B_162,rollsum_HR_162,rollsum_BB_162,rollsum_runs_162,rollsum_SB_162,rollsum_CS_162,rollsum_ERR_162,rollsum_BATAVG_162,rollsum_OBP_162,rollsum_SLG_162,rollsum_OBS_162,rollsum_AB_30,rollsum_H_30,rollsum_2B_30,rollsum_3B_30,rollsum_HR_30,rollsum_BB_30,rollsum_runs_30,rollsum_SB_30,rollsum_CS_30,rollsum_ERR_30,rollsum_BATAVG_30,rollsum_OBP_30,rollsum_SLG_30,rollsum_OBS_30
season_game,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1989022,19890430,0,Sun,CIN,NL,22,5,54,D,NaN,NaN,NaN,PHI12,43605.0,169.0,000201002,32.0,8.0,1.0,0.0,2.0,5.0,0.0,1.0,0.0,4.0,1.0,7.0,2.0,1.0,1.0,0.0,5.0,5.0,3.0,3.0,0.0,0.0,27.0,6.0,1.0,0.0,1.0,0.0,brocf901,Fred Brocklander,engeb901,Bob Engel,rungp901,Paul Runge,rennd901,Dutch Rennert,NaN,(none),NaN,(none),rosep001,Pete Rose,dibbr001,Rob Dibble,mcwil001,Larry McWilliams,franj001,John Franco,davie001,Eric Davis,browt001,Tom Browning,danik001,Kal Daniels,7.0,saboc001,Chris Sabo,5.0,larkb001,Barry Larkin,6.0,oneip001,Paul O'Neill,9.0,grifk001,Ken Griffey,3.0,winnh001,Herm Winningham,8.0,reedj001,Jeff Reed,2.0,oestr001,Ron Oester,4.0,browt001,Tom Browning,1.0,NaN,Y,1989,-2,0,8,198904300,0,PHI,5400.0,1319.0,246.0,25.0,115.0,472.0,627.0,188.0,56.0,127.0,0.244259,0.305007,0.362963,0.667970,1003.0,238.0,39.0,2.0,20.0,78.0,100.0,23.0,13.0,26.0,0.237288,0.292322,0.339980,0.632302
2007008,20070410,0,Tue,CIN,NL,8,4,65,N,NaN,NaN,NaN,PHO01,21225.0,193.0,10300000000,39.0,8.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,2.0,0.0,6.0,6.0,5.0,5.0,0.0,0.0,32.0,10.0,0.0,0.0,0.0,0.0,wegnm901,Mark Wegner,froeb901,Bruce Froemming,knigb901,Brian Knight,wintm901,Mike Winters,NaN,(none),NaN,(none),narrj001,Jerry Narron,cruzj005,Juan Cruz,weatd001,David Weathers,NaN,(none),hairs001,Scott Hairston,lohsk001,Kyle Lohse,hamij003,Josh Hamilton,8.0,philb001,Brandon Phillips,4.0,dunna001,Adam Dunn,7.0,grifk002,Ken Griffey,9.0,encae001,Edwin Encarnacion,5.0,hatts001,Scott Hatteberg,3.0,gonza002,Alex Gonzalez,6.0,valej004,Javier Valentin,2.0,lohsk001,Kyle Lohse,1.0,NaN,Y,2007,1,1,9,200704100,0,ARI,5494.0,1401.0,283.0,13.0,206.0,607.0,727.0,126.0,33.0,125.0,0.255005,0.329126,0.423735,0.752861,957.0,224.0,40.0,2.0,25.0,115.0,105.0,24.0,5.0,24.0,0.234065,0.316231,0.358412,0.674643
1994030,19940507,0,Sat,CIN,NL,30,11,51,N,NaN,NaN,NaN,CIN08,26735.0,200.0,40031030x,38.0,18.0,7.0,0.0,0.0,8.0,1.0,1.0,0.0,5.0,1.0,5.0,1.0,0.0,1.0,0.0,10.0,3.0,4.0,4.0,2.0,0.0,27.0,8.0,2.0,0.0,1.0,0.0,gormb901,Brian Gorman,ripps901,Steve Rippley,tatat901,Terry Tata,grege901,Eric Gregg,NaN,(none),NaN,(none),johnd105,Davey Johnson,smilj001,John Smiley,willb001,Brian Williams,NaN,(none),morrh001,Hal Morris,smilj001,John Smiley,larkb001,Barry Larkin,6.0,morrh001,Hal Morris,3.0,kellb002,Roberto Kelly,8.0,mitck001,Kevin Mitchell,7.0,sandr002,Reggie Sanders,9.0,fernt001,Tony Fernandez,5.0,boonb002,Bret Boone,4.0,dorsb001,Brian Dorsett,2.0,smilj001,John Smiley,1.0,NaN,Y,1994,4,1,18,199

In [13]:
# Make a dictionary that maps a team name to it's data frame
# Create the team level dataframe for each team - put in dict for easy access
team_data_dict = {}
for team in df.team_v.unique():
    team_data_dict[team] = create_team_df(team)

In [14]:
# Go through the rows of the main dataframe, and augment it with home and visiting teams' features
## Create a variety of summarized statistics for each game
## For each game, we look up the home and visiting team in the team
## data dictionary, and then look up the game, and pull the relevant stats

BATAVG_162_h = np.zeros(df.shape[0])
BATAVG_162_v = np.zeros(df.shape[0])
OBP_162_h = np.zeros(df.shape[0])
OBP_162_v = np.zeros(df.shape[0])
SLG_162_h = np.zeros(df.shape[0])
SLG_162_v = np.zeros(df.shape[0])
OBS_162_h = np.zeros(df.shape[0])
OBS_162_v = np.zeros(df.shape[0])
SB_162_h = np.zeros(df.shape[0])
SB_162_v = np.zeros(df.shape[0])
CS_162_h = np.zeros(df.shape[0])
CS_162_v = np.zeros(df.shape[0])
ERR_162_h = np.zeros(df.shape[0])
ERR_162_v = np.zeros(df.shape[0])
BATAVG_30_h = np.zeros(df.shape[0])
BATAVG_30_v = np.zeros(df.shape[0])
OBP_30_h = np.zeros(df.shape[0])
OBP_30_v = np.zeros(df.shape[0])
SLG_30_h = np.zeros(df.shape[0])
SLG_30_v = np.zeros(df.shape[0])
OBS_30_h = np.zeros(df.shape[0])
OBS_30_v = np.zeros(df.shape[0])
SB_30_h = np.zeros(df.shape[0])
SB_30_v = np.zeros(df.shape[0])
CS_30_h = np.zeros(df.shape[0])
CS_30_v = np.zeros(df.shape[0])
ERR_30_h = np.zeros(df.shape[0])
ERR_30_v = np.zeros(df.shape[0])
i=0
for index, row in df.iterrows():
    if i%2500==0:
        print(i)
    home_team = row['team_h']
    visit_team = row['team_v']
    game_index_v = row['season']*1000 + row['game_no_v']
    game_index_h = row['season']*1000 + row['game_no_h']
    BATAVG_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_BATAVG_162']
    BATAVG_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_BATAVG_162']
    OBP_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBP_162']
    OBP_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBP_162']
    SLG_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SLG_162']
    SLG_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SLG_162']
    OBS_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBS_162']
    OBS_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBS_162']
    SB_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SB_162']
    SB_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SB_162']
    CS_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_CS_162']
    CS_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_CS_162']
    ERR_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_ERR_162']
    ERR_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_ERR_162']
    BATAVG_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_BATAVG_30']
    BATAVG_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_BATAVG_30']
    OBP_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBP_30']
    OBP_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBP_30']
    SLG_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SLG_30']
    SLG_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SLG_30']
    OBS_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBS_30']
    OBS_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBS_30']
    SB_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SB_30']
    SB_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SB_30']
    CS_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_CS_30']
    CS_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_CS_30']
    ERR_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_ERR_30']
    ERR_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_ERR_30']
    i+=1

0
2500
5000
7500
10000
12500
15000
17500
20000
22500
25000
27500
30000
32500
35000
37500
40000
42500
45000
47500
50000
52500
55000
57500
60000
62500
65000
67500
70000
72500
75000
77500
80000
82500
85000
87500
90000
92500
95000
97500
100000
102500
105000
107500
110000
112500
115000


In [15]:
## We then put the constructed arrays into the main game level dataframe
df['BATAVG_162_h'] = BATAVG_162_h
df['BATAVG_162_v'] = BATAVG_162_v
df['OBP_162_h'] = OBP_162_h
df['OBP_162_v'] = OBP_162_v
df['SLG_162_h'] = SLG_162_h
df['SLG_162_v'] = SLG_162_v
df['OBS_162_h'] = OBS_162_h
df['OBS_162_v'] = OBS_162_v
df['SB_162_h'] = SB_162_h
df['SB_162_v'] = SB_162_v
df['CS_162_h'] = CS_162_h
df['CS_162_v'] = CS_162_v
df['ERR_162_h'] = ERR_162_h
df['ERR_162_v'] = ERR_162_v
df['BATAVG_30_h'] = BATAVG_30_h
df['BATAVG_30_v'] = BATAVG_30_v
df['OBP_30_h'] = OBP_30_h
df['OBP_30_v'] = OBP_30_v
df['SLG_30_h'] = SLG_30_h
df['SLG_30_v'] = SLG_30_v
df['OBS_30_h'] = OBS_30_h
df['OBS_30_v'] = OBS_30_v
df['SB_30_h'] = SB_30_h
df['SB_30_v'] = SB_30_v
df['CS_30_h'] = CS_30_h
df['CS_30_v'] = CS_30_v
df['ERR_30_h'] = ERR_30_h
df['ERR_30_v'] = ERR_30_v


df.shape

df.sample(25)

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,BATAVG_162_h,BATAVG_162_v,OBP_162_h,OBP_162_v,SLG_162_h,SLG_162_v,OBS_162_h,OBS_162_v,SB_162_h,SB_162_v,CS_162_h,CS_162_v,ERR_162_h,ERR_162_v,BATAVG_30_h,BATAVG_30_v,OBP_30_h,OBP_30_v,SLG_30_h,SLG_30_v,OBS_30_h,OBS_30_v,SB_30_h,SB_30_v,CS_30_h,CS_30_v,ERR_30_h,ERR_30_v
393,19740515,0,Wed,MON,NL,25,PHI,NL,33,5,4,54,N,NaN,NaN,NaN,PHI12,10105.0,175.0,400000001,002101000,29.0,8.0,1.0,1.0,0.0,5.0,3.0,1.0,1.0,8.0,1.0,3.0,2.0,1.0,1.0,0.0,10.0,3.0,3.0,3.0,0.0,0.0,27.0,10.0,1.0,1.0,2.0,0.0,33.0,9.0,2.0,1.0,0.0,4.0,2.0,0.0,0.0,5.0,1.0,11.0,3.0,0.0,2.0,0.0,9.0,3.0,5.0,5.0,0.0,0.0,27.0,16.0,0.0,0.0,1.0,0.0,varge901,Ed Vargo,froeb901,Bruce Froemming,mcshj901,John McSherry,rungp901,Paul Runge,NaN,(none),NaN,(none),maucg101,Gene Mauch,ozard801,Danny Ozark,walkt103,Tom Walker,scarm101,Mac Scarce,NaN,(none),singk001,Ken Singleton,renks101,Steve Renko,ruthd001,Dick Ruthven,huntr101,Ron Hunt,5.0,lintl101,Larry Lintz,6.0,daviw102,Willie Davis,8.0,singk001,Ken Singleton,9.0,fairr101,Ron Fairly,3.0,bailb103,Bob Bailey,7.0,cox-j102,Jim Cox,4.0,footb101,Barry Foote,2.0,renks101,Steve Renko,1.0,cashd101,Dave Cash,4.0,bowal001,Larry Bowa,6.0,unsed101,Del Unser,8.0,luzig001,Greg Luzinski,7.0,huttt101,Tom Hutton,3.0,schmm001,Mike Schmidt,5.0,andem101,Mike Anderson,9.0,boonb001,Bob Boone,2.0,ruthd001,Dick Ruthven,1.0,NaN,Y,1974,-1,0,9,197405150,0.256243,0.248463,0.313742,0.336349,0.381650,0.359285,0.695392,0.695633,68.0,83.0,46.0,64.0,153.0,154.0,0.261765,0.241171,0.317935,0.331556,0.369608,0.323915,0.687543,0.655471,20.0,16.0,9.0,13.0,34.0,34.0
707,20030523,0,Fri,CHN,NL,47,HOU,NL,48,5,7,51,N,NaN,NaN,NaN,HOU03,29660.0,158.0,200200001,60000010x,36.0,12.0,3.0,0.0,1.0,5.0,0.0,0.0,1.0,4.0,0.0,6.0,0.0,0.0,2.0,0.0,9.0,2.0,6.0,6.0,1.0,0.0,24.0,8.0,1.0,1.0,1.0,0.0,32.0,7.0,1.0,0.0,1.0,7.0,0.0,0.0,0.0,3.0,0.0,6.0,1.0,0.0,1.0,0.0,4.0,5.0,5.0,5.0,1.0,0.0,27.0,11.0,0.0,0.0,2.0,0.0,davig901,Gerry Davis,rapue901,Ed Rapuano,carlm901,Mark Carlson,nauep901,Paul Nauert,NaN,(none),NaN,(none),baked002,Dusty Baker,willj107,Jimy Williams,lines001,Scott Linebrink,priom001,Mark Prior,wagnb001,Billy Wagner,merco002,Orlando Merced,priom001,Mark Prior,lines001,Scott Linebrink,grudm001,Mark Grudzielanek,4.0,martr003,Ramon Martinez,6.0,pattc001,Corey Patterson,8.0,aloum001,Moises Alou,7.0,o

In [16]:
df.to_csv('df_bp1.csv', index=False)